## Управление с прогнозируюшими моделями

Управление с прогнозирующими моделями подразумевает:
* наличие внутренней динамической (прогнозирующей) модели процесса $\dot{\mathbf{x}} = \mathbf{f}(\mathbf{x},\mathbf{u},t)$

* выбор целевой функции $J$ для оптимизации на основе прогнозирующей модели на конечном интервале времени (как правило соответствующем определённому числу тактов управления $P$, $P$ называется горизонтом прогноза)

* выбор алгоритма оптимизации при помощи которого определяется минимум целевой функции $J$ по управлению $u$ на интервале времени $t \in [t_{k}, t_{k+P}]$

На каждом такте управления $t \in [t_{k}, t_{k+1}]$ решается задача оптимизации (возможно условной с любыми ограничениями на компоненты фазового вектора и управляющие параметры) после чего в качестве значения функции управления на этом такте принимается равным $\mathbf{u}_k$, полученным как $argmin J$ на первом из $P$ тактов, вошедших в решение оптимизационной задачи.

В качестве примера целевой функции может быть рассмотрена

$$J = \sum_{k=1}^P{w_{x_k}}(\mathbf{r}_k - \mathbf{x}_k)^2 + \sum_{k=1}^P{w_{u}}(\mathbf{u}_{max} - \mathbf{u}_k)^2,$$

где

$\mathbf{x}_k$ - значения вектора фазовых пременных системы на такте $k$,

$\mathbf{r}_k$ - задающий сигнал, который должна воспроизводить система управления,

$\mathbf{u}_k$ - значения вектора управляющих параметров системы на такте $k$,

$\mathbf{u}_{max}$ - значения ограничений на компоненты вектора управляющй параметров системы,

$w_{x_k}$, $w_{u}$, - вестовые коэффициенты.

## Пример. Обращенный маятник на тележке

### Обозначения
$s : смещение \ тележки\ [м]; $ 

$\alpha : угол,\ который\ образует\ маятник\ с\ вертикалью\ [rad]; $

$u  : сила,\ приложенная\ к\ тележке,\ действующая\ как\ управляющий\ вход\ [Н]; $

$m  : масса\ тележки\ [кг]; $

$M  :  масса\ маятника\ [кг]; $

$l : длина\ маятника\ [м]; $

$g : ускорение\ свободного\ падения\ [м/с^2]; $

### Уравнения Лагранжа

Кинетическая энергия
$$T = \frac{m}{2}\left[(l\dot{\alpha}\cos{\alpha}+ \dot{s})^2 + (l\dot{\alpha}\sin{\alpha})^2\right] + \frac{M}{2}\dot{s}^2$$


Потенциальная энергия
$$\Pi = mgl\cos{\alpha}$$


Уравнения Лагранжа
\begin{array}\\
ml^2\ddot{\alpha} + ml\ddot{s}\cos{\alpha} - ml\dot{s}\dot{\alpha}\sin{\alpha}- mgl\sin{\alpha} = 0 \\
ml\ddot{\alpha}\cos{\alpha} + (m + M)\ddot{s} - ml\dot{\alpha}^2\sin{\alpha} = u
\end{array}
\right. 
$$

В нормальной форме Коши уравнения обращённого маятника имеют вид:
$$\left\{ 
\begin{array}\\
\dot{\alpha} = \omega \\
\dot{\omega} = \frac{1}{(M+m\sin{\alpha}^2)l} \left[(M+m)g\sin{\alpha} - ml\omega^2\sin{\alpha}\cos{\alpha}-u\cos{\alpha}\right] \\
\dot{s} = v \\
\dot{v} = \frac{1}{(M+m\sin{\alpha}^2)} \left[-mg\sin{\alpha}\cos{\alpha} + ml\omega^2\sin{\alpha} + u\right]
\end{array}
\right. 
$$

### После упрощения и линеаризации*

\begin{equation*}
\begin{pmatrix}
  \dot{\alpha} \\ \dot{\omega} \\ \dot{s} \\ \dot{v}
\end{pmatrix} = 
 \begin{bmatrix}
0& 1& 0& 0&\\
(M + m) \cdot g & 0& 0& 0& \\
0& 0& 0& 1& \\
 - m \cdot g \cdot l& 0& 0& 0
\end{bmatrix} \cdot {\begin{pmatrix}
  \alpha \\ \omega \\ s \\ v
\end{pmatrix}}\cdot\frac{1}{M l} + {\begin{pmatrix}
  0 \\ -1 \\ 0 \\ l
\end{pmatrix}}\cdot\frac{u}{M l}
\end{equation*}

*В коде ниже  для сравнения реализованы обе модели (и полная и линеаризованная)

In [1]:
import numpy as np
import scipy as sci
from scipy.integrate import solve_ivp 
from scipy.optimize import minimize
import matplotlib.pyplot as plt

In [2]:
class Parameters(object):
    pass
sim = Parameters()
sim.time = 20
sim.step = 0.1
sim.t = np.arange(0, sim.time, sim.step)
sim.loop_count = len(sim.t)

cnst = Parameters()
cnst.g = 9.81
cnst.m = 1
cnst.M = 10
cnst.L = 1
cnst.A = np.array([[0, 1, 0, 0],
                  [ (cnst.M + cnst.m) * cnst.g, 0, 0,  0], 
                  [0, 0, 0, 1],  
                  [-cnst.m * cnst.g * cnst.L, 0, 0, 0]]) / (cnst.M * cnst.L)
cnst.b = np.array([0,  -1 , 0, cnst.L]) / (cnst.M * cnst.L)
cnst.u_max = 40

#### Структура mpc ниже содержит параметры, определяющие состав целевой функции и интервал, на котором производится решение оптимизационной задачи:

In [3]:
mpc = Parameters()
mpc.p_horizon = 10          # горизонт прогноза
mpc.constraintpenalty = 0   # штраф за превышение ограничений на управление при составлении целевой функции в оптимизации
mpc.movepenalty = 0         # штраф за изменение управления на одном шаге при составлении целевой функции в оптимизации
mpc.strongfinish = 10       # штраф за отклонение последнего значения от цели

In [4]:
def rhs(t, x, u):   
    cosA = np.cos(x[0])
    sinA = np.sin(x[0])
    p = cnst
    
    x_dot = np.zeros(4)
    
    x_dot[0] = x[1]
    x_dot[1] = ( (p.M + p.m) * p.g * sinA - p.m * p.L * x[1]**2 * sinA * cosA - u * cosA ) / (p.M + p.m * sinA**2) / p.L
    x_dot[2] = x[3]
    x_dot[3]=  ( -p.m * p.g * sinA * cosA + p.m * p.L * x[1]**2 * sinA + u ) / (p.M + p.m * sinA**2) / p.L
    
    return x_dot

def simiplified_model(t, x, u):
    return cnst.A.dot(x) + cnst.b.dot(u)

def cost_function(u_mpc):
    x_pred = np.zeros((len(u_mpc)+1, 4))
    x_pred[0, :] = states[i-1, :] 
    
    for k in range(0, len(u_mpc)):       
        time_span = [0, sim.step]
        sol = solve_ivp(lambda t, x: rhs(t, x, u_mpc[0]), time_span, x_pred[k, :])
        x_pred[k+1, :] = sol.y.T[-1]

    cost_integral = np.sum(np.square(x_pred[:, 0]))

    u_mpc_magnitude = np.abs(u_mpc)
    cost_constraint = mpc.constraintpenalty * np.sum(u_mpc_magnitude[u_mpc_magnitude > cnst.u_max])
    
    if len(u_mpc) > 1:
        cost_move = mpc.movepenalty * np.max(np.abs(np.diff(u_mpc)))
    else:
        cost_move = 0
        
    cost_finish = mpc.strongfinish * x_pred[-1, 0]**2
    
    total_cost = cost_integral + cost_constraint + cost_move + cost_finish
        
    return total_cost

Цикл по тактам управления, внутри каждого из которых решается задача оптимизации c целевой функцией $cost\_function$, определенной выше и с ограничением на управление $|u| \leq cnst.u\_max$. 

In [ ]:
u = np.zeros(sim.loop_count)
u_optimized = np.zeros(mpc.p_horizon)
states = np.zeros((sim.loop_count, 4))

# ограничения для оптимизации
c1 = ({'type':'ineq', 'fun': lambda u: -np.abs(u) + cnst.u_max})
cons = [c1]

# инициализация
states[0, :] = np.array([ np.pi * 20 / 180, 0, 0, 0])
print('cur_state:', states[0, :] )

for i in range(1, sim.loop_count):
    ts = [sim.step * (i-1), sim.step * i]
    sol_i = solve_ivp(lambda t, x: rhs(t, x, u[i]), ts, states[i-1, :])
    states[i, :] = sol_i.y.T[-1]
    print('cur_state:', states[i, :])
    
    # инициализация
    len_to_end = min(mpc.p_horizon, sim.loop_count - i)
    u_mpc0 = np.zeros(len_to_end)
    u_mpc0 = u_optimized[0: min(len_to_end, mpc.p_horizon)]
    
    solution = minimize(cost_function, u_mpc0, method='SLSQP', constraints=cons)
    u_optimized = solution.x  

    if i < sim.loop_count - 1:
        u[i+1] = u_optimized[0]

cur_state: [0.34906585 0.         0.         0.        ]
cur_state: [ 0.36745155  0.37062235 -0.00156528 -0.0314438 ]
cur_state: [0.40590866 0.4057375  0.01337876 0.32974828]
cur_state: [0.45012505 0.48679189 0.06426745 0.68742692]
cur_state: [0.50507116 0.62214139 0.15074095 1.04141264]
cur_state: [0.57680688 0.82527775 0.27242454 1.39165866]
cur_state: [0.67300811 1.11501973 0.42897083 1.7389191 ]
cur_state: [0.80347876 1.51488659 0.62020406 2.08621408]
cur_state: [0.98051994 2.05072048 0.84647575 2.44168978]
cur_state: [1.23719959 3.08305566 1.07246812 2.09004297]
cur_state: [1.59635694 4.09121363 1.26748605 1.82768746]
cur_state: [2.05218113 4.99825273 1.4414241  1.66492961]
cur_state: [2.58664518 5.61643621 1.60137465 1.52486967]
cur_state: [3.15396826 5.5984484  1.74130871 1.23618397]
cur_state: [3.68164854 4.86146942 1.84147315 0.74268979]
cur_state: [4.11546141 3.79368748 1.88809883 0.19385879]
cur_state: [ 4.43979396  2.7002601   1.88228146 -0.29754593]
cur_state: [ 4.6575991 

cur_state: [ 5.10676488e-04 -3.37732648e-04 -8.39387472e+00 -1.60901331e-01]
cur_state: [ 4.77063176e-04 -3.40568346e-04 -8.40994052e+00 -1.60413985e-01]
cur_state: [ 4.44173169e-04 -3.23136792e-04 -8.42596023e+00 -1.59979693e-01]
cur_state: [ 4.13309777e-04 -2.99672234e-04 -8.44193838e+00 -1.59582754e-01]
cur_state: [ 3.83666712e-04 -2.98511095e-04 -8.45787719e+00 -1.59193007e-01]
cur_state: [ 3.55058947e-04 -2.78780445e-04 -8.47377939e+00 -1.58850554e-01]
cur_state: [ 3.27445435e-04 -2.78447435e-04 -8.48964775e+00 -1.58516122e-01]
cur_state: [ 3.00895523e-04 -2.57317627e-04 -8.50548503e+00 -1.58229223e-01]
cur_state: [ 2.75561376e-04 -2.53913771e-04 -8.52129401e+00 -1.57949902e-01]
cur_state: [ 2.51508088e-04 -2.31470610e-04 -8.53707723e+00 -1.57714001e-01]
cur_state: [ 2.28430184e-04 -2.34230841e-04 -8.55283674e+00 -1.57475809e-01]
cur_state: [ 2.06276684e-04 -2.12816717e-04 -8.56857475e+00 -1.57284174e-01]
cur_state: [ 1.85097232e-04 -2.14574871e-04 -8.58429350e+00 -1.57090432e-01]

In [ ]:
# визуализация   
fig1 = plt.figure(figsize=(16,8))
ax1 = fig1.add_subplot(2,2,1)

ax1.set_title("Angle")
ax1.plot(sim.t, states[:, 0] * 180. / np.pi, label = r'$ \alpha $', color = 'red')
ax1.set_ylabel(r'angle, [deg]')
ax1.set_xlabel(r't, [s]')
ax1.grid(True)
ax1.legend()

ax2 = fig1.add_subplot(2,2,2)

ax2.set_title("Angular Velocity")
ax2.plot(sim.t, states[:, 1], label = '$\omega$', color = 'red')
ax2.set_ylabel(r'angular velocity, [rad/s]')
ax2.set_xlabel(r't, [s]')
ax2.grid(True)
ax2.legend()

ax3 = fig1.add_subplot(2,2,3)

ax3.set_title("Trolley Position")
ax3.plot(sim.t, states[:, 2], label = '$s$', color = 'red')
ax3.set_ylabel(r's, [m]')
ax3.set_xlabel(r't, [s]')
ax3.grid(True)
ax3.legend()

ax4 = fig1.add_subplot(2,2,4)

u_upper_bound = np.ones(np.size(sim.t)) * cnst.u_max
u_lower_bound = -u_upper_bound

ax4.set_title("Control Action")
ax4.plot(sim.t, u, label = '$u$', color = 'red')
ax4.plot(sim.t, u_upper_bound, linestyle = '--', color = 'black')
ax4.plot(sim.t, u_lower_bound, linestyle = '--', color = 'black')
ax4.set_ylabel(r'u, [Nm]')
ax4.set_xlabel(r't, [s]')
ax4.grid(True)

plt.tight_layout()

### Библиотеки, реализующие MPC

[Model predictive control python toolbox](https://www.do-mpc.com/en/latest/)

[GEKKO](https://gekko.readthedocs.io/en/latest/imode.html?highlight=mpc#mpc)

[Dynamics and control with jupyter notebooks](https://dynamics-and-control.readthedocs.io/en/latest/2_Control/7_Multivariable_control/Simple%20MPC.html)